In [ ]:
import argparse
from dataclasses import dataclass, field
from typing import List
import json
import math
import random
import time
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import d4rl # Import required to register environments
import deepdish as dd
import os
import wandb
import pickle
import pandas as pd
import time
from IPython.display import clear_output, HTML, Image, display

import sys
sys.path.append('./')
sys.path.append('./data')
sys.path.append('./..')
sys.path.append('./../human_exp')
sys.path.append('./../data')
sys.path.append('./../..')
torch.set_num_threads(4)

from reward_model import RewardModel
from reward_model import remove_outliers, compute_distribution, compute_comparable_distribution_in_bins, reject_sampling
from decision_transformer.models.decision_transformer import BidirectionalTransformer
from decision_transformer.training.contrastive_trainer import ContrativeTrainer
from env_wrapper import create_env, create_dataset, set_seed_everywhere, discount_cumsum

#### reward model for human labeling

In [2]:
class RewardModelHuman(RewardModel):
    def __init__(self, ds, da, device, trajectories, max_ep_len=500,
                 ensemble_size=3, lr=3e-4, mb_size=128, size_segment=1, activation='tanh',
                 capacity=5e5, large_batch=10,
                 teacher_beta=-1, teacher_gamma=1, 
                 teacher_eps_mistake=0, teacher_eps_skip=0, teacher_eps_equal=0,
                 use_skip_label=False, video_path='./human_exp/video_walker-walk/', 
                 segment_name_list=[], segment_traj_dict={},
                 ):
        super().__init__(ds, da, device, trajectories, max_ep_len=max_ep_len,
            ensemble_size=ensemble_size, lr=lr, mb_size=mb_size, size_segment=size_segment, activation=activation,
            capacity=capacity, large_batch=large_batch,
            teacher_beta=teacher_beta, teacher_gamma=teacher_gamma, 
            teacher_eps_mistake=teacher_eps_mistake, teacher_eps_skip=teacher_eps_skip, teacher_eps_equal=teacher_eps_equal,
            use_skip_label=use_skip_label,
        )
        self.get_label = self.get_onehot_label
        self.video_path = video_path
        self.segment_name_list = segment_name_list
        self.segment_traj_dict = segment_traj_dict
        self.segment_num = len(segment_name_list)
        print(self.segment_num)

        self.result_df = pd.DataFrame(columns=['query_idx_0', 'query_idx_1', 'return_0', 'return_1', 'rd', 'gt_label', 'human_label'])

    
    def get_queries(self, mb_size=20):
        batch_index_1 = np.random.choice(self.segment_num, size=mb_size, replace=True)
        batch_index_2 = np.random.choice(self.segment_num, size=mb_size, replace=True)
        states_1 = np.zeros((mb_size, self.size_segment, self.ds), dtype=np.float32)
        states_2 = np.zeros((mb_size, self.size_segment, self.ds), dtype=np.float32)
        actions_1 = np.zeros((mb_size, self.size_segment, self.da), dtype=np.float32)
        actions_2 = np.zeros((mb_size, self.size_segment, self.da), dtype=np.float32)
        rewards_1 = np.zeros((mb_size, self.size_segment, 1), dtype=np.float32)
        rewards_2 = np.zeros((mb_size, self.size_segment, 1), dtype=np.float32)

        for i in range(mb_size):
            # print(f'i = {i}')
            # print(self.segment_name_list[batch_index_1[i]])
            # print(self.segment_traj_dict[self.segment_name_list[batch_index_1[i]]])
            states_1[i] = np.array(self.segment_traj_dict[self.segment_name_list
                            [batch_index_1[i]]]['observations']).reshape(-1, self.ds)
            actions_1[i] = np.array(self.segment_traj_dict[self.segment_name_list
                            [batch_index_1[i]]]['actions']).reshape(-1, self.da)
            rewards_1[i] = np.array(self.segment_traj_dict[self.segment_name_list
                            [batch_index_1[i]]]['rewards']).reshape(-1, 1)
            states_2[i] = np.array(self.segment_traj_dict[self.segment_name_list
                            [batch_index_2[i]]]['observations']).reshape(-1, self.ds)
            actions_2[i] = np.array(self.segment_traj_dict[self.segment_name_list
                            [batch_index_2[i]]]['actions']).reshape(-1, self.da)
            rewards_2[i] = np.array(self.segment_traj_dict[self.segment_name_list
                            [batch_index_2[i]]]['rewards']).reshape(-1, 1)
        
        sa_t_1 = np.concatenate([states_1, actions_1], axis=2)  # Batch x size_seg x dim of s&a
        sa_t_2 = np.concatenate([states_2, actions_2], axis=2)
        r_t_1, r_t_2 = rewards_1, rewards_2  # Batch x size_seg x 1
        tr_1, tr_2 = np.zeros((mb_size, 1)), np.zeros((mb_size, 1))
        segment_name_list_0 = [self.segment_name_list[i] for i in batch_index_1]
        segment_name_list_1 = [self.segment_name_list[i] for i in batch_index_2]
        return sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1


    def get_onehot_label(self, sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, 
                         segment_name_list_0, segment_name_list_1, use_skip=True):
        '''
        Labeling rules and prompts are the same as verify_teacher.ipynb
        '''
        # print(segment_name_list_0)
        total_label_num = len(segment_name_list_0)
        label_results = []
        gt_labels = np.zeros((total_label_num, 2))

        for i in range(total_label_num):
            # 给 self.result_df 新写一行
            return_0, return_1 = np.sum(r_t_1[i]), np.sum(r_t_2[i])
            gt_labels[i] = np.array([1.0, 0]) if return_0 > return_1 else np.array([0, 1.0])
            if use_skip:
                self.result_df.loc[len(self.result_df)] = {
                    'query_idx_0': segment_name_list_0[i], 
                    'query_idx_1': segment_name_list_1[i], 
                    'return_0': return_0, 
                    'return_1': return_1, 
                    'rd': np.abs(return_0 - return_1), 
                    'gt_label': "1 0" if return_0 > return_1 else "0 1", 
                    'human_label': ""
                }

                clear_output()
                print(f"{i+1}th among total 20 feedbacks")
                query_idx_0 = segment_name_list_0[i]
                query_idx_1 = segment_name_list_1[i]
                print(f'query_idx_0: {query_idx_0}, query_idx_1: {query_idx_1}')

                segment_0 = os.path.join(self.video_path, f'{query_idx_0}.gif')
                segment_1 = os.path.join(self.video_path, f'{query_idx_1}.gif')
                print(f'segment_0: {segment_0}')
                print(f'segment_1: {segment_1}')

                time.sleep(0.1)
                display(HTML(f'''
                <div style="display: inline-block; margin-right: 100px;">
                    <img src="{segment_0}" width="400" loop="true" >
                </div>
                '''))
                time.sleep(0.1)
                display(HTML(f'''
                <div style="display: inline-block;">
                    <img src="{segment_1}" width="400" loop="true" >
                </div>
                '''))
                time.sleep(0.1)

                select = input('select')
                if select == 'quit' or select == 'exit':
                    break
                self.result_df.loc[len(self.result_df) - 1, 'human_label'] = select
                label_results.append(select)

                self.result_df.to_csv('./human_label_results.csv', index=False)
        
        if not use_skip:
            return sa_t_1, sa_t_2, r_t_1, r_t_2, gt_labels, {}

        labels = np.array([[1, 0] if '1 0' in lbl else ([0, 1] if '0 1' in lbl else [0.5, 0.5]) 
                  for lbl in label_results])
        labels_undist_count = np.sum(np.all(labels == np.array([0.5, 0.5]), axis=1))
        labels_mismatch_count = np.sum(np.all(labels != gt_labels, axis=1))
        metrics = {
            'skip_ratio': labels_undist_count / total_label_num,
            'error_ratio': (labels_mismatch_count - labels_undist_count) / total_label_num
        }
        return sa_t_1, sa_t_2, r_t_1, r_t_2, labels, metrics


    def uniform_sampling(self):
        # get queries
        sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1 = \
            self.get_queries(mb_size=self.mb_size)

        # get labels
        sa_t_1, sa_t_2, r_t_1, r_t_2, labels, metrics = self.get_label(
            sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1)
        if len(labels) > 0:
            self.put_queries(sa_t_1, sa_t_2, r_t_1, r_t_2, labels)
        
        return len(labels), metrics
    

    def disagreement_sampling(self):
        # get queries
        sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1 = \
            self.get_queries(mb_size=self.mb_size*self.large_batch)
        
        # get final queries based on uncertainty
        _, disagree = self.get_rank_probability(sa_t_1, sa_t_2)
        top_k_index = (-disagree).argsort()[:self.mb_size]
        r_t_1, sa_t_1, tr_1 = r_t_1[top_k_index], sa_t_1[top_k_index], tr_1[top_k_index]
        r_t_2, sa_t_2, tr_2 = r_t_2[top_k_index], sa_t_2[top_k_index], tr_2[top_k_index]
        segment_name_list_0 = [segment_name_list_0[i] for i in top_k_index]
        segment_name_list_1 = [segment_name_list_1[i] for i in top_k_index]
        
        # get labels
        sa_t_1, sa_t_2, r_t_1, r_t_2, labels, metrics = self.get_label(
            sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1)        
        if len(labels) > 0:
            self.put_queries(sa_t_1, sa_t_2, r_t_1, r_t_2, labels)
        
        return len(labels), metrics


    def contrastive_sampling(self):
        distances, labels = self.sample_learned_queries(self.mb_size * self.large_batch)
        idx = remove_outliers(distances)  # remove outliers
        distances, labels = distances[idx], labels[idx]
        bin_centers, hist = compute_distribution(distances, num_discrete=self.num_discrete)

        comparable = (labels[:, 0] != 0.5)
        # comparable
        dist_comparable = distances[comparable]
        hist_comparable = compute_comparable_distribution_in_bins(
            dist_comparable, bin_centers, num_discrete=self.num_discrete)
        hist_comparable_density = hist_comparable / (hist + 1e-6)
        hist_comparable_density /= np.sum(hist_comparable_density)
        # non-comparable
        dist_non_comparable = distances[~comparable]
        hist_non_comparable = compute_comparable_distribution_in_bins(
            dist_non_comparable, bin_centers, num_discrete=self.num_discrete)
        hist_non_comparable_density = hist_non_comparable / (hist + 1e-6)
        hist_non_comparable_density /= np.sum(hist_non_comparable_density)

        # get queries
        sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1 = \
            self.get_queries(mb_size=self.mb_size * self.large_batch)
        new_distances = self.convert_queries_to_embeddings(sa_t_1, sa_t_2)
        idx = remove_outliers(new_distances)  # remove outliers
        sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, new_distances = \
            sa_t_1[idx], sa_t_2[idx], r_t_1[idx], r_t_2[idx], tr_1[idx], tr_2[idx], new_distances[idx]
        segment_name_list_0 = [segment_name_list_0[i] for i in idx]
        segment_name_list_1 = [segment_name_list_1[i] for i in idx]
        
        new_hist = compute_comparable_distribution_in_bins(
            new_distances, bin_centers, num_discrete=self.num_discrete)
        target_hist_minus = np.clip((hist_comparable_density - hist_non_comparable_density), 0, None) * hist
        target_hist_minus /= np.sum(target_hist_minus)
        target_hist_div = np.clip((hist_comparable_density / (hist_non_comparable_density + 1e-6)), 0, None) * hist
        target_hist_div /= np.sum(target_hist_div)
        target_hist = (target_hist_minus + target_hist_div) * 0.5

        # reject sampling
        idx = reject_sampling(new_distances, new_hist, target_hist, bin_centers, 
                              num_discrete=self.num_discrete)
        sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, new_distances = \
            sa_t_1[idx], sa_t_2[idx], r_t_1[idx], r_t_2[idx], tr_1[idx], tr_2[idx], new_distances[idx]
        segment_name_list_0 = [segment_name_list_0[i] for i in idx]
        segment_name_list_1 = [segment_name_list_1[i] for i in idx]

        # get final queries based on uncertainty
        _, disagree = self.get_rank_probability(sa_t_1, sa_t_2)
        top_k_index = (-disagree).argsort()[:self.mb_size]
        r_t_1, sa_t_1, tr_1 = r_t_1[top_k_index], sa_t_1[top_k_index], tr_1[top_k_index]
        r_t_2, sa_t_2, tr_2 = r_t_2[top_k_index], sa_t_2[top_k_index], tr_2[top_k_index]
        segment_name_list_0 = [segment_name_list_0[i] for i in top_k_index]
        segment_name_list_1 = [segment_name_list_1[i] for i in top_k_index]
        
        # get labels
        sa_t_1, sa_t_2, r_t_1, r_t_2, labels, metrics = self.get_label(
            sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1)
        if len(labels) > 0:
            self.put_queries(sa_t_1, sa_t_2, r_t_1, r_t_2, labels)
        
        return len(labels), metrics
    

    def test_reward_model_accuracy(self):
        ''' test the reward model on the test set '''
        sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, segment_name_list_0, segment_name_list_1 = \
            self.get_queries(self.train_batch_size)
        sa_t_1, sa_t_2, r_t_1, r_t_2, labels, _ = self.get_label(
            sa_t_1, sa_t_2, r_t_1, r_t_2, tr_1, tr_2, 
            segment_name_list_0, segment_name_list_1, use_skip=False)
        ensemble_acc = np.array([0 for _ in range(self.de)])
        total = r_t_1.shape[0]
        if isinstance(labels, np.ndarray):
            labels = torch.from_numpy(labels).float().to(self.device)
        labels = torch.argmax(labels, dim=1)

        for member in range(self.de):
            # get logits
            r_hat1 = self.r_hat_member(sa_t_1, member=member)
            r_hat2 = self.r_hat_member(sa_t_2, member=member)
            r_hat1 = r_hat1.sum(axis=1)
            r_hat2 = r_hat2.sum(axis=1)
            r_hat = torch.cat([r_hat1, r_hat2], axis=-1)

            # compute acc
            _, predicted = torch.max(r_hat.data, 1)
            correct = (predicted == labels).sum().item()
            ensemble_acc[member] += correct

        ensemble_acc = ensemble_acc / total

        return np.mean(ensemble_acc)






#### config

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--env', type=str, default='walker-walk')
parser.add_argument('--dataset', type=str, default='medium-expert')
parser.add_argument('--K', type=int, default=50)  # segment size & context size, original 20
parser.add_argument('--segment_length', type=int, default=50)  # should be same as K
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--seed', type=int, default=0)
# contrastive model
parser.add_argument('--use_contrastive', type=bool, default=True)  # if True, only train reward model
parser.add_argument('--batch_size', type=int, default=64)
parser.add_argument('--embed_dim', type=int, default=256)  # original 128
parser.add_argument('--n_layer', type=int, default=4)  # original 3
parser.add_argument('--n_head', type=int, default=4)  # original 1
parser.add_argument('--activation_function', type=str, default='relu')
parser.add_argument('--dropout', type=float, default=0.1)
parser.add_argument('--learning_rate', '-lr', type=float, default=1e-4)
parser.add_argument('--z_star_lr', type=float, default=1e-3)
parser.add_argument('--weight_decay', '-wd', type=float, default=1e-4)
parser.add_argument('--warmup_steps', type=int, default=10000)
parser.add_argument('--num_steps_per_iter', type=int, default=500)  # original 2000
parser.add_argument('--num_steps_init', type=int, default=5000)  # 10 iters
# parser.add_argument('--num_steps_per_iter', type=int, default=2)  # debug
# parser.add_argument('--num_steps_init', type=int, default=2)  # 10 iters  # debug
parser.add_argument('--dist_dim', type=int, default=30)
parser.add_argument('--n_bins', type=int, default=31)
parser.add_argument('--gamma', type=float, default=1.00)
parser.add_argument('--save_model', type=bool, default=True)
parser.add_argument('--z_dim', type=int, default=16)  # original 1
# PbHIM
parser.add_argument('--similarity_fn', type=str, default='l2')
parser.add_argument('--norm_loss_ratio', type=float, default=0.1)
parser.add_argument('--comp_loss_ratio', type=float, default=0.1)
parser.add_argument('--pref_loss_ratio', type=float, default=1.0)
parser.add_argument('--pref_loss_impl', type=str, default='pairwise')
# for eval
parser.add_argument('--num_eval_episodes', type=int, default=20)
# parser.add_argument('--num_eval_episodes', type=int, default=2)  # debug
parser.add_argument('--save_rollout', type=bool, default=False)
parser.add_argument('--output_dir', type=str, default='./../results')

# reward learning
parser.add_argument('--reward_batch', type=int, default=20)
parser.add_argument('--max_feedback', type=int, default=100)
parser.add_argument('--feed_type', type=str, default='c')  # d, c (contrastive)
parser.add_argument('--train_num_iter', type=int, default=50)
parser.add_argument('--reward_lr', type=float, default=3e-4)
parser.add_argument('--reward_update', type=int, default=50)

# dummy
parser.add_argument('--dummy1', type=str, default='d')  # d, c (contrastive)
parser.add_argument('--dummy2', type=int, default=50)  # d, c (contrastive)
parser.add_argument('--dummy3', type=float, default=3e-4)  # d, c (contrastive)

args = parser.parse_args(args=[])

print(args)

# random seed
set_seed_everywhere(args.seed)

# log dir
use_contrastive = args.use_contrastive
if use_contrastive:
    save_dir = f'reward-{args.env}-{args.dataset}-conbdt'
    save_dir += f'-norm-{args.norm_loss_ratio}-comp-{args.comp_loss_ratio}-pref-{args.pref_loss_ratio}'
    save_dir += f'_fb_{args.max_feedback}_q_{args.reward_batch}_human_{args.feed_type}'
    save_dir += f'-ctx_{args.K}-seed_{args.seed}-{time.strftime("%Y%m%d-%H%M%S")}'
else:
    save_dir = f'reward-{args.env}-{args.dataset}-nocon'
    save_dir += f'_fb_{args.max_feedback}_q_{args.reward_batch}_human_{args.feed_type}'
    save_dir += f'-ctx_{args.K}-seed_{args.seed}-{time.strftime("%Y%m%d-%H%M%S")}'
output_dir = os.path.join(args.output_dir, save_dir)
os.makedirs(output_dir, exist_ok=True)

eval_dir = os.path.join(output_dir, f'eval')
os.makedirs(eval_dir, exist_ok=True)

with open(os.path.join(output_dir, 'params.json'), mode="w") as f:
    json.dump(args.__dict__, f, indent=4)

variant=vars(args)

#### 1. build env and dataset

In [ ]:
gpu = variant.get('gpu', 0)
device = torch.device(
    f"cuda:{gpu}" if (torch.cuda.is_available() and gpu >= 0) else "cpu"
)

env_name, dataset = variant['env'], variant['dataset']
seed = variant['seed']
n_bins = variant['n_bins']
use_contrastive = variant['use_contrastive']
print(f'use_contrastive: {use_contrastive}')
gamma = variant['gamma']
assert gamma == 1.
z_dim = variant['z_dim']

env, eval_env, scale, max_ep_len = create_env(env_name, seed)
state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
trajectories, states, traj_lens, returns, rewards = create_dataset(
    env_name, dataset, using_notebook=True)

# used for input normalization
states = np.concatenate(states, axis=0)
state_mean, state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6

num_timesteps = sum(traj_lens)

print('=' * 50)
print(f'Starting new experiment: {env_name} {dataset}')
print(f'{len(traj_lens)} trajectories, {num_timesteps} timesteps found')
print(f'Average return: {np.mean(returns):.2f}, std: {np.std(returns):.2f}')
print(f'Max return: {np.max(returns):.2f}, min: {np.min(returns):.2f}')
print(f'z-dim: {z_dim}')
print('=' * 50)

K = variant['K']
batch_size = variant['batch_size']

# for evaluation with best/50% trajectories
_idxes = np.argsort([np.sum(path['rewards']) for path in trajectories]) # rank 0 is the most bad demo.
trajs_rank = np.empty_like(_idxes)
trajs_rank[_idxes] = np.arange(len(_idxes))
train_indices =  [i for i in range(len(trajs_rank))]


def get_batch(batch_size=256, max_len=K):
    batch_inds = np.random.choice(
        np.array(train_indices),
        size=batch_size,
        replace=True,
    )
    s, a, r, d, rtg, timesteps, mask = [], [], [], [], [], [], []
    for i in range(batch_size):
        traj = trajectories[int(batch_inds[i])]
        si = random.randint(0, traj['rewards'].shape[0] - 1)

        s.append(traj['observations'][si:si + max_len].reshape(1, -1, state_dim))
        a.append(traj['actions'][si:si + max_len].reshape(1, -1, act_dim))
        r.append(traj['rewards'][si:si + max_len].reshape(1, -1, 1))
        if 'terminals' in traj:
            d.append(traj['terminals'][si:si + max_len].reshape(1, -1))
        else:
            d.append(traj['dones'][si:si + max_len].reshape(1, -1))
        timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
        timesteps[-1][timesteps[-1] >= max_ep_len] = max_ep_len-1  # padding cutoff
        rtg.append(discount_cumsum(traj['rewards'][si:], gamma=1.)[:s[-1].shape[1] + 1].reshape(1, -1, 1))
        if rtg[-1].shape[1] <= s[-1].shape[1]:
            rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)

        tlen = s[-1].shape[1]
        s[-1] = np.concatenate([np.zeros((1, max_len - tlen, state_dim)), s[-1]], axis=1)
        s[-1] = (s[-1] - state_mean) / state_std
        a[-1] = np.concatenate([np.ones((1, max_len - tlen, act_dim)) * -10., a[-1]], axis=1)
        r[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), r[-1]], axis=1)
        d[-1] = np.concatenate([np.ones((1, max_len - tlen)) * 2, d[-1]], axis=1)
        rtg[-1] = np.concatenate([np.zeros((1, max_len - tlen, 1)), rtg[-1]], axis=1)
        timesteps[-1] = np.concatenate([np.zeros((1, max_len - tlen)), timesteps[-1]], axis=1)
        mask.append(np.concatenate([np.zeros((1, max_len - tlen)), np.ones((1, tlen))], axis=1))

    s = torch.from_numpy(np.concatenate(s, axis=0)).to(dtype=torch.float32, device=device)  # (B, K, state_dim)
    a = torch.from_numpy(np.concatenate(a, axis=0)).to(dtype=torch.float32, device=device)
    r = torch.from_numpy(np.concatenate(r, axis=0)).to(dtype=torch.float32, device=device)  # (B, K, 1)
    d = torch.from_numpy(np.concatenate(d, axis=0)).to(dtype=torch.long, device=device)
    rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).to(dtype=torch.float32, device=device) / scale  # (B, K, 1)
    timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).to(dtype=torch.long, device=device)
    mask = torch.from_numpy(np.concatenate(mask, axis=0)).to(device=device)

    return s, a, r, d, rtg, timesteps, mask



#### 2. reward learning support

In [ ]:
# reward learning config
max_feedback = variant['max_feedback']
total_feedback, labeled_feedback = 0, 0
segment_length = variant['segment_length']
reward_lr = variant['reward_lr']
reward_batch = variant['reward_batch']
reward_update = variant['reward_update']
feed_type = variant['feed_type']

# 读取 ./human_exp/video_{env_name} 下所有以 .gif 结尾的文件，组成一个 list
video_path = f'./../human_exp/video_{env_name}/'
print("video_path: ", video_path)
segment_name_list = [f[:-4] for f in os.listdir(video_path) if f.endswith('.gif')]
segment_traj_dict = {seg_name: pickle.load(open(os.path.join(
    video_path, f'{seg_name}.pkl'), 'rb')) for seg_name in segment_name_list}
segment_num = len(segment_name_list)
print("segment_name_list: ", segment_name_list[:5])


reward_model = RewardModelHuman(
    ds=state_dim, da=act_dim, device=device, mb_size=reward_batch,
    size_segment=segment_length, trajectories=trajectories,
    max_ep_len=max_ep_len, teacher_eps_skip=0.0, lr=reward_lr,
    video_path=video_path, segment_name_list=segment_name_list, segment_traj_dict=segment_traj_dict)


def get_pref_batch(batch_size=256, max_len=K, reward_model=reward_model):
    max_pref_len = reward_model.capacity if reward_model.buffer_full else reward_model.buffer_index
    batch_inds = np.random.choice(np.arange(max_pref_len), size=batch_size, replace=True)
    sa_t_1 = reward_model.buffer_seg1[batch_inds]
    sa_t_2 = reward_model.buffer_seg2[batch_inds]
    labels = reward_model.buffer_label[batch_inds]
    states_1, actions_1 = sa_t_1[:, :, :state_dim], sa_t_1[:, :, state_dim:]
    states_2, actions_2 = sa_t_2[:, :, :state_dim], sa_t_2[:, :, state_dim:]
    rewards_1 = reward_model.buffer_reward1[batch_inds]
    rewards_2 = reward_model.buffer_reward2[batch_inds]
    dones_1 = dones_2 = torch.zeros((batch_size, max_len), dtype=torch.long, device=device)
    attention_mask_1 = torch.ones((batch_size, max_len), dtype=torch.float32, device=device)
    attention_mask_2 = torch.ones((batch_size, max_len), dtype=torch.float32, device=device)
    rtg_1 = rtg_2 = torch.ones((batch_size, max_len, 1), dtype=torch.float32, device=device)
    random_time_start = np.random.randint(0, max_ep_len - max_len - 1, size=(batch_size, 2))
    timesteps_1 = np.arange(max_len).reshape(1, -1) + random_time_start[:, 0].reshape(-1, 1)
    timesteps_2 = np.arange(max_len).reshape(1, -1) + random_time_start[:, 1].reshape(-1, 1)

    # to tensor
    states_1 = torch.from_numpy(states_1).to(dtype=torch.float32, device=device)
    actions_1 = torch.from_numpy(actions_1).to(dtype=torch.float32, device=device)
    rewards_1 = torch.from_numpy(rewards_1).to(dtype=torch.float32, device=device)
    timesteps_1 = torch.from_numpy(timesteps_1).to(dtype=torch.long, device=device)
    states_2 = torch.from_numpy(states_2).to(dtype=torch.float32, device=device)
    actions_2 = torch.from_numpy(actions_2).to(dtype=torch.float32, device=device)
    rewards_2 = torch.from_numpy(rewards_2).to(dtype=torch.float32, device=device)
    timesteps_2 = torch.from_numpy(timesteps_2).to(dtype=torch.long, device=device)
    labels = torch.from_numpy(labels).to(dtype=torch.float32, device=device)
    return states_1, actions_1, rewards_1, dones_1, rtg_1, timesteps_1, attention_mask_1, \
            states_2, actions_2, rewards_2, dones_2, rtg_2, timesteps_2, attention_mask_2, \
            labels


def learn_reward(total_feedback, labeled_feedback, first_flag=0):
    print(f'total_feedback = {total_feedback}, labeled_feedback = {labeled_feedback}')
    # get feedbacks
    if first_flag == 1:
        # if it is first time to get feedback, need to use random sampling
        labeled_queries, metrics = reward_model.uniform_sampling()
    else:
        if feed_type == 0 or feed_type == 'u':
            labeled_queries, metrics = reward_model.uniform_sampling()
        elif feed_type == 1 or feed_type == 'd':
            labeled_queries, metrics = reward_model.disagreement_sampling()
        elif feed_type == 2 or feed_type == 'c':
            labeled_queries, metrics = reward_model.contrastive_sampling()
        elif feed_type == 20 or feed_type == 'cm':
            labeled_queries, metrics = reward_model.max_ratio_sampling()
        else:
            raise NotImplementedError

    # nonlocal total_feedback, labeled_feedback
    total_feedback += reward_model.mb_size
    labeled_feedback += labeled_queries
    
    train_acc = 0
    if labeled_feedback > 0:
        # update reward
        for epoch in range(reward_update):
            train_acc, reward_loss = reward_model.train_reward()
            train_acc = np.mean(train_acc)
            if train_acc > 0.97:
                break
    eval_acc = reward_model.test_reward_model_accuracy()
    print(f"Reward function is updated!! train ACC: {train_acc:.2f}, eval ACC: {eval_acc:.2f}, epoch: {epoch}")

    metrics['reward_train_acc'] = train_acc
    metrics['reward_eval_acc'] = eval_acc
    metrics['reward_loss'] = reward_loss
    return metrics, total_feedback, labeled_feedback



#### 3. define models

In [ ]:
# define the contrastive model
model = BidirectionalTransformer(
    state_dim=state_dim,
    act_dim=act_dim,
    hidden_size=variant['embed_dim'],
    z_dim=z_dim,
    max_length=K,
    max_ep_len=max_ep_len,
    # transformer parameters
    n_layer=variant['n_layer'],
    n_head=variant['n_head'],
    n_inner=4*variant['embed_dim'],
    activation_function=variant['activation_function'],
    n_positions=1024,
    resid_pdrop=variant['dropout'],
    attn_pdrop=variant['dropout'],
)
model = model.to(device=device)
warmup_steps = variant['warmup_steps']
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=variant['learning_rate'],
    weight_decay=variant['weight_decay'],
)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lambda steps: min((steps+1)/warmup_steps, 1)
)
z_star = torch.nn.parameter.Parameter(torch.empty(z_dim, requires_grad=True, device=device))
torch.nn.init.normal_(z_star)
z_star_optimizer = torch.optim.AdamW(
    [z_star],
    lr=variant["z_star_lr"],
    weight_decay=variant['weight_decay']
)
reward_model.set_bdt_model(model)

trainer = ContrativeTrainer(
    model=model,
    optimizer=optimizer,
    batch_size=batch_size,
    get_batch=get_batch,
    get_pref_batch=get_pref_batch,
    z_star=z_star,
    z_star_optimizer=z_star_optimizer,
    scheduler=scheduler,
    loss_fn=lambda s_hat, a_hat, r_hat, s, a, r: torch.mean((a_hat - a)**2),
    eval_fns=[],
    eval_bdt_z_stars=[],
    similarity_fn=variant['similarity_fn'], 
    norm_loss_ratio=variant['norm_loss_ratio'],
    comp_loss_ratio=variant['comp_loss_ratio'],
    pref_loss_ratio=variant['pref_loss_ratio'],
    pref_loss_impl=variant['pref_loss_impl'], 
    device=device,
)

if use_contrastive:
    name = f'conbdt_{env_name}-{dataset}'
    name += f'_norm_{variant["norm_loss_ratio"]}_comp_{variant["comp_loss_ratio"]}_pref_{variant["pref_loss_ratio"]}'
    name += f'_fb_{max_feedback}_q_{reward_batch}_human_{feed_type}'
    name += f'_ctx_{K}_seed_{seed}_{time.strftime("%Y%m%d-%H%M%S")}'
else:
    name = f'reward_{env_name}-{dataset}_fb_{max_feedback}_q_{reward_batch}_human_{feed_type}'
    name += f'_ctx_{K}_seed_{seed}_{time.strftime("%Y%m%d-%H%M%S")}'
    assert feed_type != 'c'  # shouldn't use contrastive sampling
wandb.init(project='clarify_reward', name=name, config=variant)

#### 4. training

In [ ]:
# training loop
itr = 0
total_feedback, labeled_feedback = 0, 0
while total_feedback < max_feedback:
    reward_model.change_batch(1.0)
    if reward_model.mb_size + total_feedback > max_feedback:
        reward_model.set_batch(max_feedback - total_feedback)

    metrics, total_feedback, labeled_feedback = learn_reward(total_feedback, labeled_feedback, 
                                                             first_flag=1 if itr == 0 else 0)
    wandb.log(metrics, step=total_feedback)
    if use_contrastive:
        outputs = trainer.train_iteration(
            num_steps=variant['num_steps_init'] if itr == 0 else variant['num_steps_per_iter'], iter_num=itr+1, print_logs=True)
        wandb.log(outputs, step=total_feedback)
    itr += 1

    # save reward model, contrastive model
    if total_feedback % 100 == 0:
        reward_model.save_reward_model(os.path.join(output_dir, f'reward_{total_feedback}.pth'))
        if use_contrastive:
            torch.save(model.state_dict(), os.path.join(output_dir, f'dt_{total_feedback}.pth'))
            # torch.save(z_star, os.path.join(output_dir, f'z_star_{total_feedback}.pth'))

In [ ]:
wandb.finish()